In [1]:
import pandas as pd
import numpy as np
import os
import time
import warnings
warnings.filterwarnings('ignore')

# Adjust CR dataset to balanced dataset

In [2]:
df_CRTR = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_CRTR_selRFE')
df_CRTE = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_CRTE_selRFE')

In [3]:
dy_CRTR = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_CR_TR_Label')
dy_CRTE = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_CR_TE_label')

In [4]:
df_CRTR.head(1)

,Unnamed: 0,AAC_D,AAC_E,AAC_L,AAC_M,AAC_R,AAC_S,AAC_Y,APAAC_Pc1.R,APAAC_Pc1.Q,...,QSOrder_Grantham.Xd.18,QSOrder_Grantham.Xd.19,QSOrder_Grantham.Xd.20,QSOrder_Grantham.Xd.21,QSOrder_Grantham.Xd.23,QSOrder_Grantham.Xd.25,QSOrder_Grantham.Xd.26,QSOrder_Grantham.Xd.27,QSOrder_Grantham.Xd.28,QSOrder_Grantham.Xd.30
0,0,0.070155,0.048751,0.102259,0.010702,0.027943,0.065398,0.029727,46.388532,63.167363,...,0.033621,0.031979,0.034244,0.03439,0.035338,0.031261,0.034983,0.033995,0.032082,0.033454


In [5]:
dy_CRTR.head(1)

,Unnamed: 0,id,y
0,0,CPX_91662_05666_CSGID,-1


In [6]:
CRTR = df_CRTR.merge(dy_CRTR,on='Unnamed: 0')
CRTE = df_CRTE.merge(dy_CRTE,on='Unnamed: 0')

## Adjust CRTR and create new CRTR

In [7]:
CRTR.drop_duplicates()
CRTR.shape

(19791, 561)

In [8]:
CRTE.drop_duplicates()
CRTE.shape

(4947, 561)

In [9]:
CRTR.head(1)

,Unnamed: 0,AAC_D,AAC_E,AAC_L,AAC_M,AAC_R,AAC_S,AAC_Y,APAAC_Pc1.R,APAAC_Pc1.Q,...,QSOrder_Grantham.Xd.20,QSOrder_Grantham.Xd.21,QSOrder_Grantham.Xd.23,QSOrder_Grantham.Xd.25,QSOrder_Grantham.Xd.26,QSOrder_Grantham.Xd.27,QSOrder_Grantham.Xd.28,QSOrder_Grantham.Xd.30,id,y
0,0,0.070155,0.048751,0.102259,0.010702,0.027943,0.065398,0.029727,46.388532,63.167363,...,0.034244,0.03439,0.035338,0.031261,0.034983,0.033995,0.032082,0.033454,CPX_91662_05666_CSGID,-1


In [10]:
#class count
CRTR_Class_1 = CRTR[CRTR['y'] == 1]
CRTR_Class_0 = CRTR[CRTR['y'] == -1]

In [11]:
CRTR_Class_1.shape

(1234, 561)

In [12]:
CRTR_Class_0.shape

(18557, 561)

In [13]:
CRTR_Class_0_under = CRTR_Class_0.sample(4200)
CRTR_Class_0_under.shape

(4200, 561)

In [14]:
CRTR_N = pd.concat([CRTR_Class_0_under,CRTR_Class_1],axis=0)

In [15]:
CRTR_N.shape

(5434, 561)

## Adjust CRTE and create new CRTE

In [16]:
CRTE.drop_duplicates()
CRTE.shape

(4947, 561)

In [17]:
#class count
CRTE_Class_1 = CRTE[CRTE['y'] == 1]
CRTE_Class_0 = CRTE[CRTE['y'] == -1]

In [18]:
print(CRTE_Class_1.shape)
print(CRTE_Class_0.shape)

(321, 561)
(4626, 561)


In [19]:
CRTE_Class_0_under = CRTE_Class_0.sample(1100)
CRTE_Class_0_under.shape

(1100, 561)

In [20]:
CRTE_N = pd.concat([CRTE_Class_0_under,CRTE_Class_1],axis=0)
CRTE_N.shape

(1421, 561)

## Prediction based on Deep-cascade forest

In [21]:
from deepforest import CascadeForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import matthews_corrcoef

In [22]:
X = CRTR_N.iloc[:,1:-2]
Y = CRTR_N.iloc[:,-1]

In [23]:
x = CRTE_N.iloc[:,1:-2]
y = CRTE_N.iloc[:,-1]

In [24]:
X.shape,Y.shape

((5434, 558), (5434,))

In [25]:
x.shape,y.shape

((1421, 558), (1421,))

In [28]:
model_DCF = CascadeForestClassifier(random_state=0)
model_DCF.fit(X.values,Y)
y_CR_pre = model_DCF.predict(x)
print(model_DCF.score(x,y))
print(classification_report(y,y_CR_pre))
print(matthews_corrcoef(y,y_CR_pre))

[2022-04-23 07:25:42.939] Start to fit the model:
[2022-04-23 07:25:42.939] Fitting cascade layer = 0 
[2022-04-23 07:25:55.895] layer = 0  | Val Acc = 79.076 % | Elapsed = 12.957 s
[2022-04-23 07:25:55.908] Fitting cascade layer = 1 
[2022-04-23 07:26:07.090] layer = 1  | Val Acc = 79.849 % | Elapsed = 11.182 s
[2022-04-23 07:26:07.097] Fitting cascade layer = 2 
[2022-04-23 07:26:18.314] layer = 2  | Val Acc = 79.996 % | Elapsed = 11.217 s
[2022-04-23 07:26:18.321] Fitting cascade layer = 3 
[2022-04-23 07:26:29.614] layer = 3  | Val Acc = 80.180 % | Elapsed = 11.294 s
[2022-04-23 07:26:29.623] Fitting cascade layer = 4 
[2022-04-23 07:26:40.861] layer = 4  | Val Acc = 79.886 % | Elapsed = 11.238 s
[2022-04-23 07:26:40.862] Early stopping counter: 1 out of 2
[2022-04-23 07:26:40.870] Fitting cascade layer = 5 
[2022-04-23 07:26:52.146] layer = 5  | Val Acc = 79.702 % | Elapsed = 11.277 s
[2022-04-23 07:26:52.147] Early stopping counter: 2 out of 2
[2022-04-23 07:26:52.147] Handling e

# Adjust MC dataset to balanced dataset

In [60]:
df_MCTR = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_MCTR_selRFE')
df_MCTE = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_MCTE_selRFE')

In [61]:
dy_MCTR = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_MC_TR_Label')
dy_MCTE = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_MC_TE_label')

In [62]:
MCTR = df_MCTR.merge(dy_MCTR,on='Unnamed: 0')
MCTE = df_MCTE.merge(dy_MCTE,on='Unnamed: 0')

In [63]:
MCTR.drop_duplicates()
MCTE.drop_duplicates()
MCTR.shape,MCTE.shape

((4080, 186), (1020, 186))

## Adjust MCTR and create new MCTR

In [64]:
#class count
MCTR_Class_1 = MCTR[MCTR['y'] == 1]
MCTR_Class_0 = MCTR[MCTR['y'] == -1]
MCTR_Class_1.shape,MCTR_Class_0.shape

((511, 186), (3569, 186))

In [65]:
int(MCTR_Class_1.shape[0]/0.3)

1703

In [66]:
MCTR_Class_0_under = MCTR_Class_0.sample(int(MCTR_Class_1.shape[0]/0.3))
MCTR_Class_0_under.shape

(1703, 186)

In [67]:
MCTR_N = pd.concat([MCTR_Class_0_under,MCTR_Class_1],axis=0)
MCTR_N.shape

(2214, 186)

## Adjust MCTE and create new MCTE

In [68]:
#class count
MCTE_Class_1 = MCTE[MCTE['y'] == 1]
MCTE_Class_0 = MCTE[MCTE['y'] == -1]

In [69]:
print(MCTE_Class_1.shape)
print(MCTE_Class_0.shape)

(129, 186)
(891, 186)


In [70]:
int(MCTE_Class_1.shape[0]/0.3)

430

In [71]:
MCTE_Class_0_under = MCTE_Class_0.sample(int(MCTE_Class_1.shape[0]/0.3))
MCTE_Class_0_under.shape

(430, 186)

In [72]:
MCTE_N = pd.concat([MCTE_Class_0_under,MCTE_Class_1],axis=0)
MCTE_N.shape

(559, 186)

# Prediction based on Deep-cascade forest

In [73]:
X1 = MCTR_N.iloc[:,1:-2]
Y1 = MCTR_N.iloc[:,-1]

In [74]:
x1 = MCTE_N.iloc[:,1:-2]
y1 = MCTE_N.iloc[:,-1]

In [76]:
model_DCF = CascadeForestClassifier(random_state=0)
model_DCF.fit(X1.values,Y1)
y_MC_pre = model_DCF.predict(x1)
print(model_DCF.score(x1,y1))
print(classification_report(y1,y_MC_pre))
print(matthews_corrcoef(y1,y_MC_pre))

[2022-04-06 02:58:42.133] Start to fit the model:
[2022-04-06 02:58:42.133] Fitting cascade layer = 0 
[2022-04-06 02:58:45.362] layer = 0  | Val Acc = 88.573 % | Elapsed = 3.229 s
[2022-04-06 02:58:45.367] Fitting cascade layer = 1 
[2022-04-06 02:58:48.010] layer = 1  | Val Acc = 88.708 % | Elapsed = 2.643 s
[2022-04-06 02:58:48.014] Fitting cascade layer = 2 
[2022-04-06 02:58:50.741] layer = 2  | Val Acc = 88.708 % | Elapsed = 2.728 s
[2022-04-06 02:58:50.741] Early stopping counter: 1 out of 2
[2022-04-06 02:58:50.746] Fitting cascade layer = 3 
[2022-04-06 02:58:53.488] layer = 3  | Val Acc = 88.889 % | Elapsed = 2.742 s
[2022-04-06 02:58:53.492] Fitting cascade layer = 4 
[2022-04-06 02:58:56.251] layer = 4  | Val Acc = 88.844 % | Elapsed = 2.759 s
[2022-04-06 02:58:56.251] Early stopping counter: 1 out of 2
[2022-04-06 02:58:56.256] Fitting cascade layer = 5 
[2022-04-06 02:58:59.110] layer = 5  | Val Acc = 88.889 % | Elapsed = 2.855 s
[2022-04-06 02:58:59.110] Early stopping c

# Adjust CF dataset to balanced dataset

In [84]:
df_CFTR = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_CFTR_selRFE')
df_CFTE = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_CFTE_selRFE')

In [85]:
dy_CFTR = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_CF_TR_Label')
dy_CFTE = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_CF_TE_label')

In [86]:
CFTR = df_CFTR.merge(dy_CFTR,on='Unnamed: 0')
CFTE = df_CFTE.merge(dy_CFTE,on='Unnamed: 0')

In [87]:
CFTR.drop_duplicates()
CFTE.drop_duplicates()
CFTR.shape,CFTE.shape

((2184, 278), (546, 278))

In [88]:
#Adjust CFTR and create new CFTR
#class count
CFTR_Class_1 = CFTR[CFTR['y'] == 1]
CFTR_Class_0 = CFTR[CFTR['y'] == -1]
CFTR_Class_1.shape,CFTR_Class_0.shape

((1581, 278), (603, 278))

### CF dataset is balanced.

# Adjust MF dataset to balanced dataset

In [90]:
df_MFTR = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_MFTR_selRFE')
df_MFTE = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_MFTE_selRFE')

In [91]:
dy_MFTR = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_MF_TR_Label')
dy_MFTE = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_MF_TE_label')

In [92]:
MFTR = df_MFTR.merge(dy_MFTR,on='Unnamed: 0')
MFTE = df_MFTE.merge(dy_MFTE,on='Unnamed: 0')

In [93]:
MFTR.drop_duplicates()
MFTE.drop_duplicates()
MFTR.shape,MFTE.shape

((19791, 468), (4947, 468))

In [98]:
#Adjust MFTR and create new MFTR
#class count
MFTR_Class_1 = MFTR[MFTR['y'] == 1]
MFTR_Class_0 = MFTR[MFTR['y'] == -1]
MFTR_Class_1.shape,MFTR_Class_0.shape

((5769, 468), (14022, 468))

### MF dataset is balanced

# Adjust PF dataset to balanced dataset

In [94]:
df_PFTR = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_PFTR_selRFE')
df_PFTE = pd.read_csv('C:/Python/internship/Crystal_Features_new/df_PFTE_selRFE')

In [95]:
dy_PFTR = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_PF_TR_Label')
dy_PFTE = pd.read_csv('C:/Python/internship/Crystal_y_label_Csv/df_PF_TE_label')

In [96]:
PFTR = df_PFTR.merge(dy_PFTR,on='Unnamed: 0')
PFTE = df_PFTE.merge(dy_PFTE,on='Unnamed: 0')

In [97]:
PFTR.drop_duplicates()
PFTE.drop_duplicates()
PFTR.shape,PFTE.shape

((7399, 276), (1849, 276))

In [99]:
#Adjust PFTR and create new PFTR
#class count
PFTR_Class_1 = PFTR[PFTR['y'] == 1]
PFTR_Class_0 = PFTR[PFTR['y'] == -1]
PFTR_Class_1.shape,PFTR_Class_0.shape

((1840, 276), (5559, 276))

### MF dataset is balanced